In [ ]:
"""
mlops/lifecycle.py - MLflow Model Lifecycle Management.

This module provides functions for managing the model lifecycle using MLflow,
specifically for registering new model versions and promoting them to
different stages (e.g., 'Production').
"""
import os
import json
import logging
from typing import List, Optional

import mlflow
import mlflow.pyfunc
from mlflow.tracking import MlflowClient

from mlops.model import _PyfuncChatbot
from config import CONFIG

# Set up logging for this module
logger = logging.getLogger(__name__)

def get_requirements(path="requirements.txt") -> List[str]:
    """
    Reads and returns the list of pip requirements from a file.
    """
    try:
        with open(path, "r") as f:
            return [line.strip() for line in f if line.strip() and not line.startswith('#')]
    except FileNotFoundError:
        logger.warning(f"Requirements file '{path}' not found. Using hardcoded list.")
        return [
            "mlflow>=2.14.0",
            "pydantic>=2.5.0",
            "langchain-anthropic>=0.1.9",
            "langchain-openai>=0.1.7",
            "langchain-community>=0.2.0",
            "faiss-cpu>=1.7.4",
            "openai>=1.30.0",
        ]

def register_model(model_name: str) -> str:
    """
    Logs and registers a new version of the chatbot model in MLflow.

    Args:
        model_name: The name under which to register the model.

    Returns:
        The URI of the newly registered model version.
    """
    logger.info(f"Starting model registration for '{model_name}'...")
    mlflow.set_tracking_uri(CONFIG.mlflow_tracking_uri)
    mlflow.set_experiment(CONFIG.mlflow_experiment_name)

    with mlflow.start_run(run_name=f"build-{model_name}") as run:
        # Prepare artifacts (documents and adapter hint) to be packaged with the model
        os.makedirs("artifacts", exist_ok=True)
        docs_path = os.path.join("artifacts", "docs.json")
        hint_path = os.path.join("artifacts", "adapter_hint.txt")
        with open(docs_path, "w", encoding="utf-8") as f:
            json.dump(CONFIG.rag_docs, f, ensure_ascii=False, indent=2)
        with open(hint_path, "w", encoding="utf-8") as f:
            f.write(CONFIG.rag_initial_hint)

        artifacts = {"docs": docs_path, "hint": hint_path}

        # Dynamically get pip requirements
        pip_reqs = get_requirements()

        # Log the model as an MLflow pyfunc model
        mlflow.pyfunc.log_model(
            artifact_path="model",
            python_model=_PyfuncChatbot(),
            artifacts=artifacts,
            pip_requirements=pip_reqs,
        )

        # Register the logged model in the MLflow Model Registry
        model_uri = mlflow.get_artifact_uri("model")
        result = mlflow.register_model(model_uri=model_uri, name=model_name)

        logger.info(f"Successfully registered model '{result.name}' version {result.version}.")
        return f"models:/{result.name}/{result.version}"

def promote_model(model_name: str, stage: str = "Production") -> None:
    """
    Transitions the latest model version to a target stage.

    Args:
        model_name: The name of the model in the registry.
        stage: The target stage to transition the model to (e.g., 'Staging', 'Production').
    """
    logger.info(f"Promoting latest version of model '{model_name}' to stage '{stage}'...")
    client = MlflowClient()

    # Get all versions of the model
    versions = client.get_latest_versions(model_name)
    if not versions:
        logger.error("No versions found to promote.")
        raise RuntimeError("No versions found to promote.")

    # Find the latest version by version number
    latest_version = sorted(versions, key=lambda v: int(v.version))[-1]

    # Transition the model version's stage
    client.transition_model_version_stage(
        name=model_name,
        version=latest_version.version,
        stage=stage,
        archive_existing_versions=True
    )
    logger.info(f"Promoted {model_name} v{latest_version.version} to stage '{stage}'.")